# 编码分类变量
## 独热编码: 处理分类数据的标准方法

独热编码适合取值数量不是太大的分类变量。
独热编码创建新的二进制列，表示源数据每个可能的取值。
示例如下:
![](img/001.png)

## 示例

In [1]:
# 读取数据
import pandas as pd
train_data = pd.read_csv('./input/train.csv')
test_data = pd.read_csv('./input/test.csv')

# 删除目标缺失的列
train_data.dropna(axis=0, subset=['SalePrice'], inplace=True)

target = train_data.SalePrice

# 删除所有包含空数据的列
cols_with_missing = [col for col in train_data.columns if train_data[col].isnull().any()]

candidate_train_predictors = train_data.drop(['Id', 'SalePrice'] + cols_with_missing, axis=1)
candidate_test_predictors = train_data.drop(['Id'] + cols_with_missing, axis=1)

# "cardinality" 意思是一个列不同数值的个数
low_cardinality_cols = [cname for cname in candidate_train_predictors.columns if 
                       candidate_train_predictors[cname].nunique() < 10 and 
                       candidate_train_predictors[cname].dtype == "object"]

numeric_cols = [cname for cname in candidate_train_predictors.columns if
               candidate_train_predictors[cname].dtype in ['int64', 'float64']]

my_cols = low_cardinality_cols + numeric_cols
train_predictors = candidate_train_predictors[my_cols]
test_predictors = candidate_test_predictors[my_cols]

`pandas`为每列或每个`Series`设定一个数据类型(叫做`dtype`)。

`Object`表明该列包含文本。

通常使用独热编码`Object`列，因为它们通常不能被大多数模型直接使用。`Pandas`提供了一个方便的函数`get_dummies`实现独热编码。

```python
one_hot_encoded_training_predictors = pd.get_dummies(train_predictors)
```

In [2]:
train_predictors.dtypes.sample(10)

CentralAir      object
2ndFlrSF         int64
BsmtFullBath     int64
WoodDeckSF       int64
3SsnPorch        int64
MSSubClass       int64
GarageCars       int64
Heating         object
MSZoning        object
LandSlope       object
dtype: object

独热编码通常是有用处的。

可以使用类似下面代码进行比较。

In [3]:
from sklearn.model_selection import cross_val_score
from sklearn.ensemble import RandomForestRegressor

def get_mae(X, y):
    # 为了更方便使用sklearn，结果乘以-1，使得结果为正数
    return -1 * cross_val_score(RandomForestRegressor(50),
                               X, y,
                               scoring='neg_mean_absolute_error').mean()
one_hot_encoded_training_predictors = pd.get_dummies(train_predictors)
predictors_without_categoricals = train_predictors.select_dtypes(exclude=['object'])
mae_without_categoricals = get_mae(predictors_without_categoricals, target)
mae_one_hot_encoded = get_mae(one_hot_encoded_training_predictors, target)

print('Mean Absolute Error when Dropping Categoricals: ' + str(int(mae_without_categoricals)))
print('Mean Absolute Error when One-Hot Encoding: ' + str(int(mae_one_hot_encoded)))

Mean Absolute Error when Dropping Categoricals: 18608
Mean Absolute Error when One-Hot Encoding: 17915


## 应用多个文件
当使用多个文件时，由于`Scikit-learn`对列的顺序是敏感的，所以，如果训练集合测试集的列顺序错位，结果会无效。

为了保证测试集合训练集有相同的顺序，可以这样：

In [6]:
one_hot_encoded_training_predictors = pd.get_dummies(train_predictors)
one_hot_encoded_test_predictors = pd.get_dummies(test_predictors)
one_hot_encoded_test = one_hot_encoded_training_predictors.align(one_hot_encoded_test_predictors, join='left', axis=1)

- `align()` 使得两个数据集列的排列顺序一致
- `join='left'` 指明操作和SQL语法中 `left join` 一致
    - 左连接意味着，某一列只出现在一个数据集中，会保留该列的数据
    - `join='inner'` 意味着，只保留两个数据集共有的列